In [1]:
import pandas as pd 
import numpy as np
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import random
import time


In [2]:
import requests
import re
from bs4 import BeautifulSoup

In [3]:
df = pd.read_excel('QuadriMedical.xlsx')
df.dropna(subset=['Ref. '], inplace=True)
df.set_index('Ref. ', inplace=True)

In [4]:
link_ref = r'https://www.montellano.pt/pt/?_adin=11551547647'


In [17]:
df_search = pd.DataFrame()
for reference in df.index:
    price = np.nan
    link1 = fr'https://www.montellano.pt/pt/pesquisa_36.html?c=1&term={reference}'
    content = requests.get(link1)

    # Use regular expression to extract the link
    match = re.search("location='(.*)'", content.text)

    # Check if a match is found
    if match:
        link = match.group(1)
        print("Link:", link)
    else:
        price = np.nan
        print("No Price found for article: ")
        print(df.loc[reference, "Designação do artigo"])
        print('-'*20)
        continue

    response = requests.get(link)

    # Check if the request is successful
    if response.status_code == 200:
        # Do something with the response


        # Get the HTML content
        content = response.text

        
        try:
            # Create a BeautifulSoup object
            soup = BeautifulSoup(content, 'html.parser')
            # Find the element with class "current"
            current_element = soup.find("span", class_="current")

            # Get the text of the element
            price = current_element.text
            price = float(price.replace(",", ".").replace("€", ""))

            print("Current value:", price)
        except:
            price = np.nan
            print("Current value:", price)
    else:
        price = np.nan
        print("Request failed")
    nome_artigo = df.loc[reference, "Designação do artigo"]
    print(df.loc[reference, "Designação do artigo"])
    print(price)
    print('-'*20)
    data = {
        'Designacao': [nome_artigo],
        link_ref: [price]
    }
    df_temp = pd.DataFrame(data)
    df_temp.index = [reference]
    df_temp.index.name = "Reference"
    df_search = pd.concat([df_search, df_temp])

Link: https://www.montellano.pt/pt/kulzer/seringas-sopira-carpule-kulzer_p50994.html?id=36&cat=0&pc=1
Current value: 82.0
Sopira carpule seringa de aspiração
82.0
--------------------
No link found for article: 
Pinça Adson TP42 12 cm
--------------------
Link: https://www.montellano.pt/pt/kerr/boiao-toalhitas-caviwipes-160u._p43619.html?id=36&cat=0&pc=1
Current value: 7.45
CAVIWIPES - BOIAO TOALHITAS 160 Unid
7.45
--------------------
Link: https://www.montellano.pt/pt/kerr/caviwipes-toalhitas-desinf.20u-4731245-_p43618.html?id=36&cat=0&pc=1
Current value: 74.8
Caviwipes Toalhitas desinfetantes pack plano 45 ud
74.8
--------------------
No link found for article: 
Toalhetes Pop-UpBactericida, fungicida e virucida. 
--------------------
No link found for article: 
ID 212 - Desinfetante de instrumental 2,5 LT
--------------------
Link: https://www.montellano.pt/pt/durr/id-220-desinfec.-brocas-25-l-durr_p58513.html?id=36&cat=0&pc=1
Current value: 29.9
ID 220 - Limpeza e desinfecção de ma

In [16]:
df_search.to_excel('dental.xlsx')

,Designacao,https://www.montellano.pt/pt/?_adin=11551547647
Reference,,
65347466,Sopira carpule seringa de aspiração,82.00
4731160,CAVIWIPES - BOIAO TOALHITAS 160 Unid,7.45
4731245,Caviwipes Toalhitas desinfetantes pack plano 4...,74.80
CDI220C6150,ID 220 - Limpeza e desinfecção de material rot...,29.90
CDS110P6150,OROTOL PLUS - Limpeza e desinfecção Sistemas a...,36.10
UB100,Aspirador de saliva moldável Hygoformic Bio (1...,108.97
17211,Luvas exame látex com pó - UNID,58.71
835/100,ESCOVAS PROFILAXIA 835 acabado/polido nylon 10...,107.26
16190,MOLDEIRAS DUPLAS medias lime 100 ud,103.56


In [7]:
# driver = webdriver.Chrome(ChromeDriverManager().install())

# driver.add_argument('--headless')



# wait = WebDriverWait(driver, 10)
# df_search = pd.DataFrame()

# for reference in df.index:
#     driver.get(link)
#     rand_int = random.randint(1, 10)
#     time.sleep(rand_int)
#     search_bar = wait.until(EC.presence_of_element_located((By.ID, "search")))
#     search_bar.clear()
#     search_bar.send_keys(reference)
#     search_bar.send_keys(Keys.RETURN)
#     try:
#         price_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "current")))
#         price = float(price_element.text.replace(",", ".").replace("€", ""))
#     except:
#         price = np.nan
#     nome_artigo = df.loc[reference, "Designação do artigo"]
#     print(df.loc[reference, "Designação do artigo"])
#     print(price)
#     print('-'*20)
#     data = {
#         link: [price],
#         'Designacao': [nome_artigo]
#     }
#     df_temp = pd.DataFrame(data)
#     df_temp.index = [reference]
#     df_temp.index.name = "Reference"
#     df_search = pd.concat([df_search, df_temp])
#     driver.get(r'https://portalhashtag.com/')
    
# driver.quit()